In [0]:
#https://www.leafly.com/strains/lists/condition/inflammation

!pip install requests-html > /dev/null

In [0]:
from pygments import highlight
from pygments.lexers import HtmlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
from urllib.request import urlopen
from requests.adapters import HTTPAdapter
import requests
import re
import json



In [0]:
def pretty_print_html(html_string):
  html_string = BeautifulSoup(html_string, 'html.parser').prettify()
  pygments_css = pygments_css=HtmlFormatter().get_style_defs('.highlight')
  data = highlight(html_string, HtmlLexer(), HtmlFormatter())
  return display(HTML(f'<style>{pygments_css}</style>{data}'))

In [4]:
!wget 'https://www.leafly.com/strains/lists/condition/inflammation'

--2020-03-06 01:49:10--  https://www.leafly.com/strains/lists/condition/inflammation
Resolving www.leafly.com (www.leafly.com)... 104.18.202.76, 104.18.203.76, 2606:4700::6812:cb4c, ...
Connecting to www.leafly.com (www.leafly.com)|104.18.202.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘inflammation’

inflammation            [  <=>               ] 188.71K   568KB/s    in 0.3s    

2020-03-06 01:49:12 (568 KB/s) - ‘inflammation’ saved [193239]



In [0]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://www.leafly.com/strains/lists/condition/inflammation')

In [6]:
r

<Response [200]>

In [7]:
r.html.html

'<!DOCTYPE html><html lang="en"><head><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-standard-thin.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-standard-medium.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-condensed-medium.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="dns-prefetch" href="https://public.leafly.com"/><link rel="dns-prefetch" href="https://public.leafly.com"/><script type="text/javascript" src="https://public.leafly.com/scripts/split-10.10.0.min.js" defer=""></script><script type="text/javascript" src="https://public.leafly.com/cephalopod/dist/vendors.87e2f9ad0784ea148b7c.js" defer=""></script><script type="text/javascript" src="https://public.leafly.com/cephalopod/dist/helpers.c20c4673027656787dc7.js" defer=""></script><script t

In [8]:
pretty_print_html(r.html.html)

In [0]:
#naming our html document
document = r.html

In [10]:
#finding each strain name
strain_name = document.find('.strain-tile')
strain_name

[<Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/avi-dekel'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/cbd-mango-haze'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/grand-doggy-purps'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/cbd-kush'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/in-the-pines'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/cbd-blue-shark'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/critical-widow'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/afghani-cbd'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/ringos-gift'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/haleighs-hope'>,
 <Element 'a' class=('strain-tile', '

In [0]:
#selecting the first strain name 
first_strain_name = strain_name[0]

In [12]:
#pretty print of the strain name
pretty_print_html(first_strain_name.html)

In [13]:
#pinpointed the text of the strain name!
first_strain_name.find('.strain-tile__name', first = True).text

'Avi-Dekel'

In [14]:
#figured out how to get the text only
first_strain_name.find('span', first = True).text

'Focused'

In [15]:
#pulling the attribute of the link only
first_strain_name.attrs['href']

'/strains/avi-dekel'

In [0]:
#getting ready to create the dataframe
strain_list = []
for strain in strain_name:
  name = strain.find('.strain-tile__name')[0].text
  affect = strain.find('span')[0].text
  link = strain.attrs['href']
  strain_list.append({'Name':name, 'Affect':affect, 'Links': link})

In [17]:
df = pd.DataFrame(strain_list)
df

,Name,Affect,Links
0,Avi-Dekel,Focused,/strains/avi-dekel
1,CBD Mango Haze,Focused,/strains/cbd-mango-haze
2,Grand Doggy Purps,Relaxed,/strains/grand-doggy-purps
3,CBD Kush,Focused,/strains/cbd-kush
4,In The Pines,Focused,/strains/in-the-pines
5,CBD Blue Shark,Relaxed,/strains/cbd-blue-shark
6,Critical Widow,Uplifted,/strains/critical-widow
7,Afghani CBD,Sleepy,/strains/afghani-cbd
8,Ringo's Gift,Focused,/strains/ringos-gift
9,Haleigh’s Hope,Relaxed,/strains/haleighs-hope


In [18]:
df['Links'] = df['Links'].apply(lambda x: '<a href="http://www.leafly.com/{0}">Learn More'.format(x))

HTML(df.to_html(escape=False))

,Name,Affect,Links
0,Avi-Dekel,Focused,Learn More
1,CBD Mango Haze,Focused,Learn More
2,Grand Doggy Purps,Relaxed,Learn More
3,CBD Kush,Focused,Learn More
4,In The Pines,Focused,Learn More
5,CBD Blue Shark,Relaxed,Learn More
6,Critical Widow,Uplifted,Learn More
7,Afghani CBD,Sleepy,Learn More
8,Ringo's Gift,Focused,Learn More
9,Haleigh’s Hope,Relaxed,Learn More


In [19]:
df

,Name,Affect,Links
0,Avi-Dekel,Focused,"<a href=""http://www.leafly.com//strains/avi-de..."
1,CBD Mango Haze,Focused,"<a href=""http://www.leafly.com//strains/cbd-ma..."
2,Grand Doggy Purps,Relaxed,"<a href=""http://www.leafly.com//strains/grand-..."
3,CBD Kush,Focused,"<a href=""http://www.leafly.com//strains/cbd-ku..."
4,In The Pines,Focused,"<a href=""http://www.leafly.com//strains/in-the..."
5,CBD Blue Shark,Relaxed,"<a href=""http://www.leafly.com//strains/cbd-bl..."
6,Critical Widow,Uplifted,"<a href=""http://www.leafly.com//strains/critic..."
7,Afghani CBD,Sleepy,"<a href=""http://www.leafly.com//strains/afghan..."
8,Ringo's Gift,Focused,"<a href=""http://www.leafly.com//strains/ringos..."
9,Haleigh’s Hope,Relaxed,"<a href=""http://www.leafly.com//strains/haleig..."


In [0]:
df.to_csv('inflammation_strain_list')